In [5]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from tqdm.notebook import tqdm
import torch.nn.functional as F
import GPUtil
import random
%cd colorpoem/prototypes

[WinError 3] 지정된 경로를 찾을 수 없습니다: 'colorpoem/prototypes'
C:\Users\green\Desktop\workspace\colorpoem\prototypes


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=8)
# model = torch.load("model.pt")

In [8]:
dataframe = pd.read_csv('../resource/emotion_classification.csv').dropna(axis=0)
#중립0 행복1 슬픔2 공포3 분노4 놀람5 혐오6 상처7
emodict = {'중립':0, '행복':1, '슬픔':2, '공포':3, '분노':4, '놀람':5, '혐오':6, '상처':7}
for k,v in emodict.items():
    dataframe = dataframe.replace(k, v)

dataframe.drop(dataframe[dataframe['감정_대분류']==6].index, inplace=True)
dataframe.drop(dataframe[dataframe['감정_대분류']==7].index, inplace=True)
dataframe[:10]

,사람문장1,감정_대분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,1
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,5
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,1
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,1
5,이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...,3
6,사십 대에 접어들면서 머리카락이 많이 빠져 고민이야.,2
7,이제 돈이라면 지긋지긋해.,4
8,친구들이 나를 괴롭혀. 부모님과 선생님께 얘기했는데도 믿어주지 않아.,4
9,친구 때문에 눈물 나.,2


In [9]:
dataframe['감정_대분류'].value_counts()

0    60577
2    28430
4    22794
5    19875
1    18148
3    16912
Name: 감정_대분류, dtype: int64

In [10]:
train_df, test_df = train_test_split(dataframe, test_size=0.2)

In [11]:
max_len = 128
batch_size = 64
learning_rate = 1e-5
epochs = 3
seed_val = 42

In [12]:
class MyDataset(Dataset):
    def __init__(self, df):
        total_list = []
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            text = row[0]
            encoded_list = tokenizer.encode(text, add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True)
            total_list.append(encoded_list)
        self.df = pd.DataFrame({'사람문장' : total_list, '감정_대분류' :df['감정_대분류']}).reset_index(drop=True)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = np.array(self.df.iloc[idx, 0])
        label = self.df.iloc[idx, 1]
        return text, label

In [13]:
train_dataset = MyDataset(train_df)
test_dataset = MyDataset(test_df)

  0%|          | 0/133388 [00:00<?, ?it/s]

  0%|          | 0/33348 [00:00<?, ?it/s]

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [15]:
next(iter(train_dataloader))

[tensor([[   101,   9576, 119022,  ...,      0,      0,      0],
         [   101,   9246, 119049,  ...,      0,      0,      0],
         [   101,   9781,  17196,  ...,      0,      0,      0],
         ...,
         [   101,   9651,  40419,  ...,      0,      0,      0],
         [   101,   9847, 119420,  ...,      0,      0,      0],
         [   101,   9340, 119216,  ...,      0,      0,      0]],
        dtype=torch.int32),
 tensor([5, 0, 4, 0, 1, 4, 4, 2, 0, 5, 5, 2, 0, 5, 1, 0, 0, 0, 0, 2, 0, 5, 0, 1,
         2, 5, 0, 4, 3, 0, 2, 4, 2, 0, 4, 1, 2, 4, 0, 2, 4, 3, 0, 3, 2, 0, 3, 2,
         1, 5, 0, 4, 1, 2, 4, 0, 0, 4, 2, 3, 4, 3, 0, 4])]

In [16]:
device = torch.device("cuda:0")
model.to(device)

AttributeError: 'collections.OrderedDict' object has no attribute 'to'

In [ ]:
optimizer = Adam(model.parameters(), lr=learning_rate)
p_itr = 500

In [ ]:
def categorical_accuracy(logits, label):
    top_logits = torch.argmax(F.softmax(logits, dim=1), dim=1)
    # print(top_logits)
    # print(label)
    correct = top_logits.eq(label).sum()
    return correct

In [ ]:
for epoch in range(epochs):
    # GPUtil.showUtilization()
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch_id, (text, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        sample = text.to(device)
        labels = label.to(device)
        outputs = model(sample, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]
        loss.backward()
        optimizer.step()

        acc = categorical_accuracy(logits, labels) 
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        if itr % p_itr == 0:
            print('[Epoch {}/{}] itr {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, batch_id+1, epoch_loss/(batch_id+1), epoch_acc/((1+batch_id)*batch_size)))
        #torch.save(model.state_dict(), './model.pt')
     
    model.eval()
    test_acc = 0
    for batch_id, (text, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        sample = text.to(device)
        labels = label.to(device)
        with torch.no_grad():    
            outputs = model(sample, labels=labels)
        logits = outputs[1]
        acc = categorical_accuracy(logits, labels)
        test_acc += acc.item()
    print("epoch {} test acc {}".format(epoch+1, test_acc/(batch_size*(batch_id+1))))

  0%|          | 0/2085 [00:00<?, ?it/s]

[Epoch 1/5] itr 500 -> Train Loss: 1.4584, Accuracy: 0.456
[Epoch 1/5] itr 1000 -> Train Loss: 1.2969, Accuracy: 0.522
[Epoch 1/5] itr 1500 -> Train Loss: 1.2162, Accuracy: 0.554
[Epoch 1/5] itr 2000 -> Train Loss: 1.1687, Accuracy: 0.571


  0%|          | 0/522 [00:00<?, ?it/s]

epoch 1 test acc 0.641882183908046


  0%|          | 0/2085 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6734015804597702


  0%|          | 0/2085 [00:00<?, ?it/s]

[Epoch 5/5] itr 8500 -> Train Loss: 0.7582, Accuracy: 0.725
[Epoch 5/5] itr 9000 -> Train Loss: 0.7648, Accuracy: 0.723
[Epoch 5/5] itr 9500 -> Train Loss: 0.7647, Accuracy: 0.723
[Epoch 5/5] itr 10000 -> Train Loss: 0.7668, Accuracy: 0.723


  0%|          | 0/522 [00:00<?, ?it/s]

epoch 5 test acc 0.6761554118773946


In [ ]:
def predict(sentence):
    df = pd.DataFrame({'사람문장':[sentence], '감정_대분류':[0]})
    dataset = MyDataset(df)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    
    model.eval()
    for batch_id, (text, label) in tqdm(enumerate(dataloader), total=len(dataloader)): 
        sample = text.to(device)
        with torch.no_grad():    
            outputs = model(sample)
        logits = outputs[0]
        return [k for k, v in emodict.items() if v == torch.argmax(logits)][0]

In [ ]:
predict("나 진짜 왜 살까")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

'슬픔'

In [ ]:
torch.save(model.state_dict(), './model.pt')

##### 